In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time
from multiprocessing import Process, Pool
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc

In [2]:
from process import map_feature

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
TEMP = OUTPUT + 'temp/'
MAP  = OUTPUT + 'map/'

In [4]:
# original input
df_train = pd.read_csv(FOLDER+'train.csv')
# df_test = pd.read_csv(FOLDER+'test.csv')
mols_train = df_train['molecule_name'].unique()

In [5]:
DATA_DATE = '20190721'
MAP_DATE = '20190728'
GEN_DATE = '20190817'

In [6]:
# input files
map_train = MAP + MAP_DATE + '_' + 'map_train.pickle'
map_test = MAP + MAP_DATE + '_' + 'map_test.pickle'

first_train = OUTPUT + DATA_DATE + '_' + 'features_train.pickle'
first_test = OUTPUT + DATA_DATE + '_' + 'features_test.pickle'

merged_models =  OUTPUT + MODEL_DATE + '_' + 'merged_nn_{}.h5'

In [7]:
# output

In [8]:
df_map_train = pd.read_pickle(map_train)
df_map_train_idx = df_map_train.set_index('id')

In [9]:
df_map_train.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,id,molecule_name
0,0,-1,-1,-1,-1,-1,-1,1,2,3,...,-1,-1,-1,-1,-1,-1,-1,-1,0,dsgdb9nsd_000001
1,4,-1,-1,-1,-1,-1,-1,6,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,4,dsgdb9nsd_000001
2,7,-1,-1,-1,-1,-1,-1,8,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,7,dsgdb9nsd_000001
3,9,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,9,dsgdb9nsd_000001
0,1,0,-1,-1,-1,-1,-1,2,3,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,dsgdb9nsd_000001


In [10]:
bond_types = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']

In [11]:
with open(first_train, 'rb') as f:
    features_train = pickle.load(f)

In [12]:
df_train_idx = df_train.set_index('id') 

In [18]:
def get_sc(df_idx, idx_arr):
    return df_idx.iloc[idx_arr]['scalar_coupling_constant'].values

In [19]:
def gen_merged_features(df_map_idx, features, idx_arr):
    num_ids = len(idx_arr)
    merged_features = np.zeros([num_ids, 49, 215])
    for j in range(num_ids):
        pickup_ids = df_map_idx.loc[j].values[:-1]
        for i, p in enumerate(pickup_ids):
            if p == -1:
                continue
            merged_features[j, i, :] = features[p]
    return merged_features

In [22]:
def data_flow(df_idx, df_map_idx, features, idx, batch_size=32):
    while True:
        idx_arr = random.sample(idx, batch_size)
        inputs = gen_merged_features(df_map_idx, features, idx_arr)
        input_list = []
        num_input = inputs.shape[1]
        for i in range(num_input):
            input_list.append(inputs[:,i,:])
        targets = get_sc(df_idx, idx_arr)
        yield input_list, targets